In [2]:
import logging
import time
import csv
import json
import tweepy
import re
import operator
import threading
import numpy as np
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import datetime
from dateutil import parser
import pandas as pd
from RNN_political_analysis import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from twython import Twython
import random
# enable logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s %(levelname)s %(module)s - %(funcName)s: %(message)s', 
                    datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# authorize the app to access Twitter on our behalf
consumer_key = 'vNqAPuXUzBj4TIyYgcC5pWJiD'
consumer_secret = 'uVSKRvPdn43l4L8W9RuCBbib9kQf1cxM09QzHGVYh2gnnCIPN5'
access_token = '2473317248-ocHKkHNI4juaWg8lZrbKkWtP2vyrejN6YrjnrRa'
access_secret = '6Yut2IvNLLXEI81oo5LuNuVelAAIidEe9BTDRgZHFrJLX'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)
twitter = Twython(
    consumer_key, consumer_secret,
    access_token, access_secret)


C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def set_interval(func, sec):
    def func_wrapper():
        set_interval(func, sec)
        func()
    t = threading.Timer(sec, func_wrapper)
    t.start()
    return t

model = load_model('RNN_polotical_analysis_model.h5')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 128)           192000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           131584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 128)           512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 12802     
Total params: 336,898
Trainable params: 336,642
Non-trainable params: 256
_________________________________________________________________
No

In [4]:
# establish open connection to streaming API
class MyListener(StreamListener):

    def on_data(self, data):
        try:
            tweet = parse_tweet(data)
            content = extract_content(tweet)
            with open('TwitterData.csv', 'a') as f: 
                writer = csv.writer(f, quotechar = '"')
                writer.writerow(content)
                #logger.info(content[3])
                data = tweet['text']
                #date = tweet['CREATED_AT']
                rtcount = tweet['TWEET_TYPE']
                d_t = datetime.datetime.now()
                predict_data(data, str(d_t))
      
                    
        except BaseException as e:
            logger.warning(e)

        return True

    def on_error(self, status):
        logger.warning(status)
        return True

In [5]:
# parse data
def parse_tweet(data):

    # load JSON item into a dict
    tweet = json.loads(data)


    # check if tweet is valid
    if 'user' in tweet.keys():
        # parse date    
        tweet['CREATED_AT'] = parser.parse(tweet['created_at'])

        # classify tweet type based on metadata
        if 'retweeted_status' in tweet:
            tweet['TWEET_TYPE'] = 'retweet'

        elif len(tweet['entities']['user_mentions']) > 0:
            tweet['TWEET_TYPE'] = 'mention'

        else:
            tweet['TWEET_TYPE'] = 'tweet'

        return tweet

    else:
        logger.warning("Imcomplete tweet: %s", tweet)


In [6]:
# extract relevant data to write to CSV
def extract_content(tweet):

    content = [tweet['CREATED_AT'].strftime('%Y-%m-%d'),
               tweet['text'].encode('unicode_escape'),tweet['id'],
              tweet['TWEET_TYPE']]
    #print(post)
    return content

In [7]:
counter = 0
data=[]
def predict_data(post, date):
    global counter, model, data
    post = post.lower()
    hashtag=re.findall(r"#(\w+)", post)
    post = post.replace('RT','')
    post = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",post).split())
    data.append(post)
    data=np.array(data)
    data=data.tolist()
    max_word_length = 1500
    test_tokenizer = Tokenizer(num_words=max_word_length, split=' ')
    test_tokenizer.fit_on_texts(data)
    X = test_tokenizer.texts_to_sequences(data)
    X = pad_sequences(X, maxlen=50)
    result = model.predict(X[0].reshape(1,50),batch_size=1)[0]
    positivity = round(result[1], 2)
    negativity =round(result[0], 2)
    sentiment =0
    data = []

    if positivity > 0.55:
        sentiment = positivity

    else:
        sentiment = negativity
    write_sentimet = open('sentimentData.csv','a')
    write_sentimet.write(str(date)+','+str(sentiment))
    write_sentimet.write('\n')
    write_sentimet.close()
    if len(hashtag)!=0:
        with open("hashtags.csv",'a') as resultFile:
            wr = csv.writer(resultFile, dialect='excel')
            wr.writerow(hashtag)
    data=[]

In [12]:

ids={}

                
def refresh_top_posts():
    tags={}
    with open('TwitterData.csv', newline='\n') as dataFile:
        reader = csv.reader(dataFile, delimiter=',', quotechar = '"')
        for row in reader:
            if row[2] in ids:
                val = ids[row[2]]+1
                update_id = {row[2]:val}
                ids.update(update_id)
            else:
                new_id = {row[2]:0}
                ids.update(new_id)
        
        tp = sorted(ids.items(), key=lambda x:-x[1])[:10]
        write_post = open('top_posts.csv','w')

        for x in tp:
            write_post.write(str(x[0])+','+str(x[1]))
            write_post.write('\n')
           
        write_post.close()
#refresh_tags = set_interval(refresh_tags, 15)
refresh_posts = set_interval(refresh_top_posts, 15)

In [11]:
def start_stream():
    
    while True:

        logger.warning("Twitter API Connection opened")

        try:
            twitter_stream = Stream(auth, MyListener())
            twitter_stream.filter(track=['trump','clinton'], locations=[-124.848974, 24.396308, -66.885444, 49.384358])

        except Exception as e: 
            logger.warning(e)
            continue

        finally:
            logger.warning("Twitter API Connection closed")

            

if __name__ == '__main__':
    start_stream()

2018-04-11 22:45:54 WARNING <ipython-input-11-77cf057b0d81> - start_stream: Twitter API Connection opened
2018-04-11 22:46:01 WARNING <ipython-input-5-1e7dccc9b4c6> - parse_tweet: Imcomplete tweet: {'limit': {'track': 7, 'timestamp_ms': '1523466059891'}}
2018-04-11 22:46:01 WARNING <ipython-input-4-d6ae56a28a9b> - on_data: 'NoneType' object is not subscriptable
2018-04-11 22:46:01 WARNING <ipython-input-5-1e7dccc9b4c6> - parse_tweet: Imcomplete tweet: {'limit': {'track': 15, 'timestamp_ms': '1523466059915'}}
2018-04-11 22:46:01 WARNING <ipython-input-4-d6ae56a28a9b> - on_data: 'NoneType' object is not subscriptable
2018-04-11 22:46:02 WARNING <ipython-input-5-1e7dccc9b4c6> - parse_tweet: Imcomplete tweet: {'limit': {'track': 1, 'timestamp_ms': '1523466059923'}}
2018-04-11 22:46:02 WARNING <ipython-input-4-d6ae56a28a9b> - on_data: 'NoneType' object is not subscriptable
2018-04-11 22:46:02 WARNING <ipython-input-5-1e7dccc9b4c6> - parse_tweet: Imcomplete tweet: {'limit': {'track': 10, 'ti

2018-04-11 22:46:29 WARNING <ipython-input-4-d6ae56a28a9b> - on_data: 'NoneType' object is not subscriptable
2018-04-11 22:46:30 WARNING <ipython-input-5-1e7dccc9b4c6> - parse_tweet: Imcomplete tweet: {'limit': {'track': 51, 'timestamp_ms': '1523466070892'}}
2018-04-11 22:46:30 WARNING <ipython-input-4-d6ae56a28a9b> - on_data: 'NoneType' object is not subscriptable
2018-04-11 22:46:30 WARNING <ipython-input-5-1e7dccc9b4c6> - parse_tweet: Imcomplete tweet: {'limit': {'track': 49, 'timestamp_ms': '1523466070915'}}
2018-04-11 22:46:30 WARNING <ipython-input-4-d6ae56a28a9b> - on_data: 'NoneType' object is not subscriptable
2018-04-11 22:46:30 WARNING <ipython-input-5-1e7dccc9b4c6> - parse_tweet: Imcomplete tweet: {'limit': {'track': 52, 'timestamp_ms': '1523466070871'}}
2018-04-11 22:46:30 WARNING <ipython-input-4-d6ae56a28a9b> - on_data: 'NoneType' object is not subscriptable
2018-04-11 22:46:30 WARNING <ipython-input-5-1e7dccc9b4c6> - parse_tweet: Imcomplete tweet: {'limit': {'track': 33

KeyboardInterrupt: 

3